# **Project Name**  - Vaccination Data Analysis and Visualization

##### **Project Type**   - Python,SQL,EDA,Power BI
##### **Contribution**   - Individual
##### **Team Member 1 -**  Gade Pavan Kumar Reddy

# **Project Summary -**

This project centers on analyzing global vaccination and infectious disease data to support better public health decisions. By designing and implementing a structured MySQL database, we integrated diverse datasets covering vaccination coverage, disease incidence, vaccine schedules, and country-level details. The cleaned and transformed data—comprising hundreds of thousands of records—are loaded into a relational database and connected to Power BI for advanced analytics and visualization.

Using Power BI, we developed interactive dashboards that enable users to explore trends in vaccination rates and disease incidence, uncover regional disparities, and evaluate the effectiveness of immunization programs. Scatter plots reveal the relationship between vaccination coverage and disease reduction, while geographical heatmaps identify regions of concern and progress. Trend lines, KPI indicators, and slicers facilitate dynamic filtering by year, country, or disease, making the analysis accessible and actionable for health officials and policymakers.

The solution supports scenario-based investigations—such as tracking the impact of new vaccine introductions, identifying regions for resource allocation, and monitoring progress toward international immunization targets. The system is set up for scheduled refreshes, ensuring that stakeholders always work with up-to-date information. This project offers a scalable and adaptable blueprint for turning raw health data into actionable insights, empowering public health interventions and resource prioritization for maximum impact.

# **GitHub Link**

Link- https://github.com/pavangade31/Vaccination-data-and-visualization-project

# **Problem Statement**

Despite ongoing vaccination efforts, many regions face persistent challenges in achieving high immunization coverage and controlling vaccine-preventable diseases. Fragmented data sources and a lack of integrated analytical tools make it difficult for health agencies to identify coverage gaps, evaluate program effectiveness, and respond rapidly to emerging outbreaks. Without a centralized, real-time analysis platform, decision-makers struggle to understand where interventions are most needed and to monitor progress toward health objectives.

This project addresses the need for an end-to-end data pipeline and interactive analytics—transforming disparate vaccination and disease data into unified, actionable dashboards to enable more targeted, effective public health decisions and interventions.

# ***Let's Begin !***

## Step 1: Imports & Configuration

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine,text
import warnings
warnings.filterwarnings('ignore')

## Step 2: Load Excel Data

In [2]:
coverage_df = pd.read_excel('coverage-data.xlsx')                  
incidence_df = pd.read_excel('incidence-rate-data.xlsx')                
cases_df = pd.read_excel('reported-cases-data.xlsx')                    
intro_df = pd.read_excel('vaccine-introduction-data.xlsx')              
schedule_df = pd.read_excel('vaccine-schedule-data.xlsx') 

## Step 3: Data Cleaning and Imputation

### 3.1: Checking Shape of all dataframes

In [3]:
dfs = {
    "coverage_df": coverage_df,
    "incidence_df": incidence_df,
    "cases_df": cases_df,
    "intro_df": intro_df,
    "schedule_df": schedule_df
}

for name, df in dfs.items():
    print(f"Shape of {name}:")
    print(df.shape)
    print("---------------------------------------")

Shape of coverage_df:
(399859, 11)
---------------------------------------
Shape of incidence_df:
(84946, 8)
---------------------------------------
Shape of cases_df:
(84870, 7)
---------------------------------------
Shape of intro_df:
(138321, 6)
---------------------------------------
Shape of schedule_df:
(8053, 12)
---------------------------------------


### 3.2: Checking Percentage of Missing Values in all dataframes

In [4]:
for name, df in dfs.items():
    print(f"Missing values percentage for every column in {name}:")
    print(df.isna().mean()*100)
    print("---------------------------------------")

Missing values percentage for every column in coverage_df:
GROUP                             0.000000
CODE                              0.000250
NAME                              0.318862
YEAR                              0.000250
ANTIGEN                           0.000250
ANTIGEN_DESCRIPTION               0.000250
COVERAGE_CATEGORY                 0.000250
COVERAGE_CATEGORY_DESCRIPTION     0.000250
TARGET_NUMBER                    80.235533
DOSES                            80.161257
COVERAGE                         42.360432
dtype: float64
---------------------------------------
Missing values percentage for every column in incidence_df:
GROUP                   0.000000
CODE                    0.001177
NAME                    0.001177
YEAR                    0.001177
DISEASE                 0.001177
DISEASE_DESCRIPTION     0.001177
DENOMINATOR             0.001177
INCIDENCE_RATE         27.502178
dtype: float64
---------------------------------------
Missing values percentage for ever

### 3.3: Cleaning and Imputing Coverage Data

In [5]:
df = coverage_df.copy()
df = df.drop(columns=['TARGET_NUMBER', 'DOSES'])

# Impute 'NAME'
df['NAME'] = df.groupby('CODE')['NAME'].transform(lambda x: x.ffill().bfill())
df['NAME'].fillna('Unknown', inplace=True)

# Function to impute with median only if median > 0, else keep NaN for second fill
def impute_nonzero_median(x):
    med = x.median()
    if med > 0:
        return x.fillna(med)
    else:
        return x

df['COVERAGE'] = df.groupby(['CODE', 'ANTIGEN'])['COVERAGE'].transform(impute_nonzero_median)

# Fill remaining NaNs with overall median excluding zeros
overall_median = df.loc[df['COVERAGE'] > 0, 'COVERAGE'].median()
df['COVERAGE'] = df['COVERAGE'].fillna(overall_median)

# Drop rows with any remaining missing data
df.dropna(inplace=True)

coverage_df_clean = df.copy()

**Remove**: **TARGET_NUMBER** (80.24% missing) and **DOSES** (80.16% missing) – too much missingness for reliable imputation.


Impute: NAME (0.32%) with forward-fill/backward-fill by 'CODE'; COVERAGE (42.36%) with median by group (still valuable for correlation/trend
).

All other columns have negligible missingness—fill or drop remaining NaNs.

### 3.4: Cleaning and Imputing Incidence Data

In [6]:
# --- Cleaning incidence_df ---
df_incidence = incidence_df.copy()

# Fill 'NAME' missing by forward/backward fill within 'CODE'
df_incidence['NAME'] = df_incidence.groupby('CODE')['NAME'].transform(lambda x: x.ffill().bfill())
df_incidence['NAME'].fillna('Unknown', inplace=True)

# Function to impute incidence_rate only if group's median > 0
def impute_nonzero_median(x):
    med = x.median()
    if med > 0:
        return x.fillna(med)
    else:
        return x

df_incidence['INCIDENCE_RATE'] = df_incidence.groupby('DISEASE')['INCIDENCE_RATE'].transform(impute_nonzero_median)

# Fill remaining NaNs with overall median excluding zeros
overall_median_incidence = df_incidence.loc[df_incidence['INCIDENCE_RATE'] > 0, 'INCIDENCE_RATE'].median()
df_incidence['INCIDENCE_RATE'] = df_incidence['INCIDENCE_RATE'].fillna(overall_median_incidence)

# Drop any rows still having missing values
df_incidence.dropna(inplace=True)

incidence_df_clean = df_incidence.copy()

**Remove**: None—the only column with significant missingness is INCIDENCE_RATE (27.50%), still useful for analysis.


Impute: INCIDENCE_RATE by median per 'DISEASE'; other columns use forward-fill/backward-fill by 'CODE'.

### 3.5: Cleaning and Imputing Cases Data

In [7]:
# --- Cleaning cases_df ---
df_cases = cases_df.copy()

# Fill 'NAME' missing by forward/backward fill within 'CODE'
df_cases['NAME'] = df_cases.groupby('CODE')['NAME'].transform(lambda x: x.ffill().bfill())
df_cases['NAME'].fillna('Unknown', inplace=True)

# Impute CASES only if group's median > 0
df_cases['CASES'] = df_cases.groupby('DISEASE')['CASES'].transform(impute_nonzero_median)

# Fill remaining NaNs with overall median excluding zeros
overall_median_cases = df_cases.loc[df_cases['CASES'] > 0, 'CASES'].median()
df_cases['CASES'] = df_cases['CASES'].fillna(overall_median_cases)

# Drop rows with missing data
df_cases.dropna(inplace=True)

cases_df_clean = df_cases.copy()

**Remove**: None—CASES missingness (22.86%) is high, but still usable for reduction analysis.

Impute: CASES by median per 'DISEASE'; fill others as above.

### 3.6: Cleaning and Imputing Vaccine Introduction Data

In [8]:
df = intro_df.copy()
for col in ['COUNTRYNAME', 'WHO_REGION', 'YEAR', 'DESCRIPTION', 'INTRO']:
    df[col] = df[col].ffill().bfill()
df.dropna(inplace=True)
intro_df_clean = df.copy()

**Remove/Impute**: Very low missingness (<1%); forward-fill/backward-fill all missing categorical columns (by 'ISO_3_CODE' or globally). Drop if any missing remain.

### 3.7: Cleaning and Imputing Schedule Data

In [9]:
df = schedule_df.copy()
df = df.drop(columns=['TARGETPOP'])  # remove due to high missingness

# Fill others by group
for col in ['COUNTRYNAME', 'WHO_REGION', 'YEAR', 'VACCINECODE', 'VACCINE_DESCRIPTION', 'SCHEDULEROUNDS', 'TARGETPOP_DESCRIPTION', 'GEOAREA']:
    df[col] = df.groupby('ISO_3_CODE')[col].transform(lambda x: x.ffill().bfill())
# Impute AGEADMINISTERED and SOURCECOMMENT
df['AGEADMINISTERED'].fillna('Not specified', inplace=True)
df['SOURCECOMMENT'].fillna('No comment', inplace=True)
df.dropna(inplace=True)
schedule_df_clean = df.copy()

**Remove**: TARGETPOP (52.87%)—over half missing, drop for robust analysis.

Impute: AGEADMINISTERED (12.99%) and SOURCECOMMENT (36.19%)—impute missing values with placeholders; other columns impute by ffill/bfill by 'ISO_3_CODE'.

### 3.8: Changing 'YEAR' Column into Integer 

In [10]:
coverage_df_clean['YEAR'] = coverage_df_clean['YEAR'].astype(int)
incidence_df_clean['YEAR'] = incidence_df_clean['YEAR'].astype(int)
cases_df_clean['YEAR'] = cases_df_clean['YEAR'].astype(int)
intro_df_clean['YEAR'] = intro_df_clean['YEAR'].astype(int)
schedule_df_clean['YEAR'] = schedule_df_clean['YEAR'].astype(int)

### 3.9: Checking the shape of Cleaned Dataframes

In [11]:
dfs_new = {
    "coverage_df": coverage_df_clean,
    "incidence_df": incidence_df_clean,
    "cases_df": cases_df_clean,
    "intro_df": intro_df_clean,
    "schedule_df": schedule_df_clean
}

for name, df in dfs_new.items():
    print(f"Shape of {name} after cleaning and imputation:")
    print(df.shape)
    print("---------------------------------------")

Shape of coverage_df after cleaning and imputation:
(399858, 9)
---------------------------------------
Shape of incidence_df after cleaning and imputation:
(84945, 8)
---------------------------------------
Shape of cases_df after cleaning and imputation:
(84869, 7)
---------------------------------------
Shape of intro_df after cleaning and imputation:
(138321, 6)
---------------------------------------
Shape of schedule_df after cleaning and imputation:
(8052, 11)
---------------------------------------


### 3.10: Checking Percentage of Missing Values in all Cleaned dataframes

In [12]:
for name, df in dfs_new.items():
    print(f"Missing values percentage for every column in {name} after cleaning and imputation:")
    print(df.isna().mean()*100)
    print("---------------------------------------")

Missing values percentage for every column in coverage_df after cleaning and imputation:
GROUP                            0.0
CODE                             0.0
NAME                             0.0
YEAR                             0.0
ANTIGEN                          0.0
ANTIGEN_DESCRIPTION              0.0
COVERAGE_CATEGORY                0.0
COVERAGE_CATEGORY_DESCRIPTION    0.0
COVERAGE                         0.0
dtype: float64
---------------------------------------
Missing values percentage for every column in incidence_df after cleaning and imputation:
GROUP                  0.0
CODE                   0.0
NAME                   0.0
YEAR                   0.0
DISEASE                0.0
DISEASE_DESCRIPTION    0.0
DENOMINATOR            0.0
INCIDENCE_RATE         0.0
dtype: float64
---------------------------------------
Missing values percentage for every column in cases_df after cleaning and imputation:
GROUP                  0.0
CODE                   0.0
NAME                  

### 3.11: Checking Datatypes of columns in all Cleaned dataframes

In [13]:
for name, df in dfs_new.items():
    print(f"Data types of every column in {name} after cleaning and imputation:")
    print(df.dtypes)
    print("---------------------------------------")

Data types of every column in coverage_df after cleaning and imputation:
GROUP                             object
CODE                              object
NAME                              object
YEAR                               int32
ANTIGEN                           object
ANTIGEN_DESCRIPTION               object
COVERAGE_CATEGORY                 object
COVERAGE_CATEGORY_DESCRIPTION     object
COVERAGE                         float64
dtype: object
---------------------------------------
Data types of every column in incidence_df after cleaning and imputation:
GROUP                   object
CODE                    object
NAME                    object
YEAR                     int32
DISEASE                 object
DISEASE_DESCRIPTION     object
DENOMINATOR             object
INCIDENCE_RATE         float64
dtype: object
---------------------------------------
Data types of every column in cases_df after cleaning and imputation:
GROUP                   object
CODE                    obj

### 3.12: Creating Country, Antigen, Disease dataframes 

In [14]:
# Prepare country_df_clean with columns: 'iso_code', 'name'
country_df_clean = coverage_df[['CODE', 'NAME']].drop_duplicates().rename(
    columns={
        'CODE': 'iso_code',
        'NAME': 'name'
    }
).reset_index(drop=True)

# Prepare antigen_df_clean with columns: 'antigen_id', 'description'
antigen_df_clean = coverage_df[['ANTIGEN', 'ANTIGEN_DESCRIPTION']].drop_duplicates().rename(
    columns={
        'ANTIGEN': 'antigen_id',
        'ANTIGEN_DESCRIPTION': 'description'
    }
).reset_index(drop=True)

# Prepare disease_df_clean by combining unique diseases from incidence_df and cases_df
disease_incidence = incidence_df[['DISEASE', 'DISEASE_DESCRIPTION']].drop_duplicates()
disease_cases = cases_df[['DISEASE', 'DISEASE_DESCRIPTION']].drop_duplicates()

disease_df_clean = pd.concat([disease_incidence, disease_cases]).drop_duplicates().rename(
    columns={
        'DISEASE': 'disease_code',
        'DISEASE_DESCRIPTION': 'description'
    }
).reset_index(drop=True)

### 3.13: Renaming columns in all Dataframes

In [15]:
# coverage_df_clean
coverage_df_clean = coverage_df_clean.rename(columns={
    'CODE': 'country_code',
    'YEAR': 'year',
    'ANTIGEN': 'antigen_id',
    'ANTIGEN_DESCRIPTION': 'antigen_description',
    'COVERAGE_CATEGORY': 'coverage_category',
    'COVERAGE_CATEGORY_DESCRIPTION': 'coverage_category_description',
    'COVERAGE': 'coverage'
})

# incidence_df_clean
incidence_df_clean = incidence_df_clean.rename(columns={
    'CODE': 'country_code',
    'YEAR': 'year',
    'DISEASE': 'disease_code',
    'DISEASE_DESCRIPTION': 'disease_description',
    'DENOMINATOR': 'denominator',
    'INCIDENCE_RATE': 'incidence_rate'
})

# cases_df_clean (reported_cases table)
cases_df_clean = cases_df_clean.rename(columns={
    'CODE': 'country_code',
    'YEAR': 'year',
    'DISEASE': 'disease_code',
    'DISEASE_DESCRIPTION': 'disease_description',
    'CASES': 'cases'
})

# intro_df_clean (vaccine_introduction table)
intro_df_clean = intro_df_clean.rename(columns={
    'ISO_3_CODE': 'country_code',
    'WHO_REGION': 'who_region',
    'YEAR': 'year',
    'DESCRIPTION': 'description',
    'INTRO': 'introduced'
})

# schedule_df_clean (vaccine_schedule table)
schedule_df_clean = schedule_df_clean.rename(columns={
    'ISO_3_CODE': 'country_code',
    'WHO_REGION': 'who_region',
    'YEAR': 'year',
    'VACCINECODE': 'vaccine_code',
    'VACCINE_DESCRIPTION': 'vaccine_description',
    'SCHEDULEROUNDS': 'schedule_rounds',
    'TARGETPOP_DESCRIPTION': 'target_population_description',
    'GEOAREA': 'geoarea',
    'AGEADMINISTERED': 'age_administered',
    'SOURCECOMMENT': 'source_comment'
})

### 3.14: Dropping Null values and Duplicate Values in Country, Antigen and Disease Dataframes

In [35]:
# Drop rows with null values in key columns
country_df_clean = country_df_clean.dropna(subset=['iso_code', 'name']).copy()
antigen_df_clean = antigen_df_clean.dropna(subset=['antigen_id', 'description']).copy()
disease_df_clean = disease_df_clean.dropna(subset=['disease_code', 'description']).copy()

# Drop duplicate rows based on primary keys, keep first occurrence
country_df_clean = country_df_clean.drop_duplicates(subset=['iso_code','name'], keep='first').copy()
antigen_df_clean = antigen_df_clean.drop_duplicates(subset=['antigen_id'], keep='first').copy()
disease_df_clean = disease_df_clean.drop_duplicates(subset=['disease_code'], keep='first').copy()

### 3.15: Dropping columns from all dataframes

In [27]:
coverage_df_clean = coverage_df_clean.drop(columns=['GROUP', 'NAME'], errors='ignore')

incidence_df_clean = incidence_df_clean.drop(columns=['GROUP', 'NAME'], errors='ignore')

cases_df_clean = cases_df_clean.drop(columns=['GROUP', 'NAME'], errors='ignore')

intro_df_clean = intro_df_clean.drop(columns=['COUNTRYNAME'], errors='ignore')

schedule_df_clean = schedule_df_clean.drop(columns=['COUNTRYNAME'], errors='ignore')

To ensure that only relevant columns are loaded into the database and used in analysis, unnecessary columns were removed from each cleaned DataFrame.

By applying the drop method with errors='ignore', these lines ensure that the specified columns are removed if present, and the process continues smoothly even if they are missing in certain DataFrames. This step helps maintain a clean and consistent schema across all datasets prior to database insertion and analysis.

### 3.16: Mapping 'introduced' column with boolean values

In [29]:
intro_df_clean['introduced'] = intro_df_clean['introduced'].map({'Yes': 1, 'No': 0})

## Step 4: Connecting to MySQL Database and loading data

In [39]:
# MySQL connection details
username = 'root'
password = 'root'
host = 'localhost'
port = '3306'
database = 'vaccination_data'

engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}:{port}/{database}')

tables_to_load = {
    'country': country_df_clean,
    'antigen': antigen_df_clean,
    'disease': disease_df_clean,
    'coverage': coverage_df_clean,
    'incidence': incidence_df_clean,
    'reported_cases': cases_df_clean,
    'vaccine_introduction': intro_df_clean,
    'vaccine_schedule': schedule_df_clean
}

with engine.begin() as conn:
    conn.execute(text("SET FOREIGN_KEY_CHECKS=0;"))  # Disable FK checks to allow truncation
    
    # Truncate tables before loading to avoid duplicates
    for table_name in tables_to_load.keys():
        conn.execute(text(f"TRUNCATE TABLE {table_name};"))
    
    # Insert data with append mode, since schema exists
    for table_name, df in tables_to_load.items():
        if not df.empty:
            df.to_sql(name=table_name, con=conn, if_exists='append', index=False)
            print(f"Inserted {len(df)} records into {table_name}")
        else:
            print(f"Skipping empty DataFrame for {table_name}")
    
    conn.execute(text("SET FOREIGN_KEY_CHECKS=1;"))  # Re-enable FK checks

Inserted 241 records into country
Inserted 69 records into antigen
Inserted 13 records into disease
Inserted 399858 records into coverage
Inserted 84945 records into incidence
Inserted 84869 records into reported_cases
Inserted 138321 records into vaccine_introduction
Inserted 8052 records into vaccine_schedule
